<a href="https://colab.research.google.com/github/eobianyor/Project3/blob/main/DemoRun_IdahoFiresModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [37]:
# =============================================================== #
# IMPORT DEPENDENCIES
# =============================================================== #
# FOR DATA
# =============================================================== #
import pandas as pd
# import os
import csv
import math
import numpy as np
from numpy import logspace
import time
import datetime
# import requests
# import datefinder

# # FOR SQL LITE
# =============================================================== #
# from sqlalchemy import create_engine
# from datetime import date

# # FOR PLOTTING
# =============================================================== #
import matplotlib.pyplot as plt
from matplotlib import style
style.use("fivethirtyeight")
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# FOR MODELING
# =============================================================== #
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# FOR KNN AND OTHER MODELS
# =============================================================== #
from scipy.optimize import curve_fit
# from splinter import Browser
# from bs4 import BeautifulSoup as BS
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# # FOR TF MODELS
# =============================================================== #
# import keras.models
# import keras.layers
# import keras.utils
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

In [38]:
# IMPORT LIBRARIES FOR COLAB
# =========================================================
#               Python Libraries for Colab
# =========================================================
import sys
# ---------------------------------------------------------
from google.colab import drive

# =========================================================
#               Mount Google Drive to Colab
# =========================================================
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


LOAD IDAHO FIRE MODEL

In [39]:
# =============================================================== #
# TEST MODEL WITH NEW DATA
# =============================================================== #
# LOAD MODEL via pickle
# =============================================================== #
import pickle

with open('/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/gridModel_model2_v1.pkl', 'rb') as modelload:
    loadedgridModel = pickle.load(modelload)

In [40]:
# =============================================================== #
# MAKE A SAMPLE ARRAY FOR DEMO
# =============================================================== #
xInput = {'avg_temp': [0],
            'prcp': [0.03],
            'humidity':[40],
            'dew_point': [-1.460],
            'stream_level':[0.001],
            'lightning_strikes': [0]}

X_input_Demo_df = pd.DataFrame(data=xInput)

X_input_Demo_df

# =============================================================== #
# Look at data type
# =============================================================== #
# type(newInput)
# newInput.dtypes

,avg_temp,prcp,humidity,dew_point,stream_level,lightning_strikes
0,0,0.03,40,-1.46,0.001,0


In [41]:
# =============================================================== #
# PREP X VALUES FOR MODEL
# =============================================================== #
# # RESHAPE X FROM DF TO ARRAY
newInput = X_input_Demo_df.values.reshape(-1, 6)

# # SCALE NEW INPUTS
newInputScaler = StandardScaler().fit(newInput)

# # CREATE VARIABLES TO HOLD SCALED DATA
scaledNewInputs = newInputScaler.transform(newInput)


# # # # View output
# print(f"Shape of X Input is {newInput.shape}")
# print(f"Type of X input is {type(newInput)}")

# =============================================================== #
# RUN TEST
# =============================================================== #
newPredictions = loadedgridModel.predict(scaledNewInputs)
newProbability = loadedgridModel.predict_proba(scaledNewInputs)

# =============================================================== #
# PRINT RESULTS
# =============================================================== #
fireProbabilityArray = np.round(newProbability [:, 1], 4)
fireProbabilityPercentage = (fireProbabilityArray[0] * 100)

print(f"The probability of a fire given these conditions is: {fireProbabilityPercentage}%")

# # =============================================================== #
# # CALCULATE CLASSIFICATION REPORT
# # =============================================================== #
# from sklearn.metrics import classification_report
# print(classification_report(scaledNewInputs, newPredictions,
#                             target_names=["True(Fire risk)", "False(No fire risk)"]))


The probability of a fire given these conditions is: 48.3%
